In [ ]:
from IPython.lib.display import YouTubeVideo

# Roadmap para futuros estudos

## Literatura

1. [Deep Learning with Python - François Chollet](https://www.amazon.com.br/Deep-Learning-Python-Francois-Chollet/dp/1617294438)
2. [Redes Neurais: Princípios e Prática - Simon Haykin](https://www.amazon.com.br/Redes-Neurais-Princ%C3%ADpios-Simon-Haykin/dp/8573077182)
3. [Deep Learning - Goodfellow; Bengio; Courville](https://www.amazon.com.br/Deep-Learning-Ian-Goodfellow/dp/0262035618)
4. [Neural Networks and Deep Learning - Michael Nielsen](http://neuralnetworksanddeeplearning.com/index.html)

## Apoio

In [ ]:
YouTubeVideo('OmJ-4B-mS-Y', width=1024, height=590)

In [ ]:
YouTubeVideo('aircAruvnKk', width=1024, height=590)

In [ ]:
YouTubeVideo('IHZwWFHWa-w', width=1024, height=590)

In [ ]:
YouTubeVideo('Ilg3gGewQ5U', width=1024, height=590)

In [ ]:
YouTubeVideo('tIeHLnjs5U8', width=1024, height=590)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D, Conv2D
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.datasets import mnist

import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
plt.figure(figsize=(10, 10))

for i in range(10):

    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(X_train[i], cv2.COLOR_GRAY2RGB))

plt.show()

In [ ]:
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
nb_filters = 32
kernel_size = (3, 3)
pool_size = (2, 2)

model = Sequential()

model.add(Conv2D(nb_filters, kernel_size, padding='same', input_shape=input_shape, activation='relu', name='Conv2DLayer1'))
model.add(Conv2D(nb_filters, kernel_size, padding='same',activation='relu', name='Conv2DLayer2'))
model.add(MaxPooling2D(pool_size=pool_size, name='MaxPoolLayer3'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units=128, activation='relu', name='DenseLayer4'))
model.add(Dropout(0.5))
model.add(Dense(units=nb_classes, activation='softmax', name='OutputLayer4'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
epochs = 5

checkpoint = ModelCheckpoint(filepath='model.hdf5', monitor='val_accuracy', verbose=True, save_best_only=True)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=128, validation_split=0.1, verbose=1, callbacks=[checkpoint])

In [ ]:
model.load_weights('model.hdf5')

In [ ]:
model.save('model.h5')

In [ ]:
model.layers[:3]

In [ ]:
output_camadas = [camada.output for camada in model.layers[:3]]

In [ ]:
activations = Model(inputs=model.input, outputs=output_camadas)

In [ ]:
# Código adaptado de Deep Learning With Python; François Chollet; Pg 164
def model_layers_output(imagem):

    activations_output = activations.predict([[imagem]])
    
    layers = list()
    
    for layer in model.layers[:3]:
        layers.append(layer.name)

    grid_imgs = 16

    for layer_name, activation_output in zip(layers, activations_output):
        features = activation_output.shape[-1]
        activation_size = activation_output.shape[1]
        numb_cols = features // grid_imgs
        display_grid = np.zeros((activation_size * numb_cols, grid_imgs * activation_size))
        
        for col in range(numb_cols):
            for row in range(grid_imgs):
                heatmap = activation_output[0, :, :, col * grid_imgs + row]
                heatmap -= heatmap.mean()
                heatmap /= heatmap.std()
                heatmap *= 64
                heatmap += 128
                heatmap = np.clip(heatmap, 0, 255)
                display_grid[col * activation_size : (col + 1) * activation_size, row * activation_size : (row + 1) * activation_size] = heatmap
                
        scale = 1. / activation_size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.axis('off')
        plt.imshow(display_grid, aspect='auto')

In [ ]:
model_layers_output(X_test[0])

In [ ]:
model_layers_output(X_test[10])

In [ ]:
model_layers_output(X_test[90])

In [ ]:
model_layers_output(X_test[100])